In [1]:
%matplotlib inline


# phonemes = {0:'i',1:'ee', 2:'oo', 3:'oo', 4:'ai', 5:'oi', 6:'i', 7:'e', 8:'er'
#       , 9:'er', 10:'oor', 11:'air', 12:'ere', 13:'ou', 14:'a', 15:'u', 16:'o'
#       , 17:'ar', 18:'ow', 19:'ou', 20:'p', 21:'f', 22:'th', 23:'t', 24:'s'
#       , 25:'sh', 26:'ch', 27:'c', 28:'b', 29:'v', 30:'th', 31:'d', 32:'z'
#       , 33:'s', 34:'j',35:'g', 36:'m', 37:'n', 38:'ng', 39:'h', 40:'w'
#       , 41:'l', 42:'r', 43:'y'}

def otherWay(phenome):
    for i,p in phonemes.iteritems():
        if p == phenome:
            return i
        
        
#phonemes = {0: 'ee', 1:'t', 2:'r', 3:'c',4:'a'}

phonemes = {0: 'i', 1:'ee', 2:'oo', 3:'oo',4:'ai', 5:'oi', 6:'i', 7:'e', 8:'er', 9:'oor', 10:'air', 11:'ere', 12:'ou',13:'a', 14:'u', 15:'o', 16:'ar', 17:'ow', 18:'ou'}



In [2]:
from scipy.io import wavfile
from scipy import fftpack
import scipy
import numpy as np
import matplotlib.pyplot as plt

rate, data = wavfile.read("1Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/7)*rate)
f = fftpack.fftfreq(int(N) ,1.0/rate)
trainingX = []


for i in range(7):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    #plt.plot(F)
    
    trainingX.append(F)
    
rate, data = wavfile.read("2Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/6)*rate)
f = fftpack.fftfreq(int(N) ,1.0/rate)


for i in range(6):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    #plt.plot(F)
    
    trainingX.append(F)
    
    
rate, data = wavfile.read("3Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/6)*rate)
f = fftpack.fftfreq(int(N) ,1.0/rate)


for i in range(6):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    #plt.plot(F)
    
    trainingX.append(F)
    

    
# rate, data = wavfile.read("cat.wav")
# seconds = (data.shape[0]/float(rate))
# N = int(.42666667*rate)


# for i in range(3):
#     subdata = data[(i)*N:(i+1)*N-1]
#     F=abs(fftpack.fft(subdata))
#     f = f[:len(F)/2]
#     F = F[:len(F)/2]
#     F = F[30:350]
#     F = F/100000
#     plt.plot(F)
#     trainingX.append(F)
# trainingX =  np.array(trainingX)

#trainingYB = np.array([otherWay('t'),otherWay('r'), otherWay('ee'),otherWay('c'),otherWay('a'), otherWay('t')])

trainingYB = np.array([otherWay('i'),otherWay('ee'), otherWay('oo'),otherWay('oo'),otherWay('ai'), otherWay('oi'), otherWay('i'), otherWay('e'), otherWay('er'), otherWay('oor'), otherWay('air'), otherWay('ere'), otherWay('ou'), otherWay('a'), otherWay('u'), otherWay('o'), otherWay('ar'), otherWay('ow'), otherWay('ou')])

trainingY = []
for i in trainingYB:
    t = np.zeros(19)-.99
    t[i] = .99
    trainingY.append(t)

trainingY = np.array(trainingY)



In [3]:

class neuralNetwork:
    
    
    def __init__(self, inodes, hnodes, onodes, lr):
        
        self.iNodes = inodes
        self.hNodes = hnodes
        self.oNodes = onodes

        self.iHidden = np.random.normal(0.0, pow(self.hNodes, -0.5), (self.hNodes, self.iNodes))
        self.hOuter = np.random.normal(0.0, pow(self.oNodes, -0.5), (self.oNodes, self.hNodes))

        
        self.lr = lr
        
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        

    
    def train(self, inp, tar):
        
        inputs = np.array(inp, ndmin=2).T
        targets = np.array(tar, ndmin=2).T
        
        
        hiddenInputs = np.dot(self.iHidden, inputs)
        hiddenOutputs = self.activation_function(hiddenInputs)
        
        
        finalInputs = np.dot(self.hOuter, hiddenOutputs)
        finalOutputs = self.activation_function(finalInputs)
        
        outputE = targets - finalOutputs
        hiddenE = np.dot(self.hOuter.T, outputE) 
        
        self.hOuter += self.lr * np.dot((outputE * finalOutputs * (1.0 - finalOutputs)), np.transpose(hiddenOutputs))
        self.iHidden += self.lr * np.dot((hiddenE * hiddenOutputs * (1.0 - hiddenOutputs)), np.transpose(inputs))
        
        

    
    def query(self, inp):
        
        inputs = np.array(inp, ndmin=2).T
        
        
        hiddenInputs = np.dot(self.iHidden, inputs)
        hiddenOutputs = self.activation_function(hiddenInputs)
        
        finalInputs = np.dot(self.hOuter, hiddenOutputs)
        finalOutputs = self.activation_function(finalInputs)
        
        return finalOutputs

In [6]:
input_nodes = 800  # frequency range 100-900
hidden_nodes = 450
output_nodes = 19 # number of phonemes trained

learning_rate = 0.0007

n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [7]:
c = []

#for l in range(10):
    
    
for i in range(100):
    for j in range(19):
        n.train(trainingX[j],trainingY[j])

# for k in range(23):
#     print(phonemes[np.argmax(n.query(trainingX[k]))])

count = 0
    
print()

rate, data = wavfile.read("1Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/7)*rate)
cat = []

for i in range(7):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    cat.append(F)
cat =  np.array(cat)

for k in range(7):
    print(phonemes[np.argmax(n.query(cat[k]))], phonemes[count])
    count = count+1
    
print()
    
rate, data = wavfile.read("2Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/6)*rate)
cat = []

for i in range(6):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    cat.append(F)
cat =  np.array(cat)

for k in range(6):
    print(phonemes[np.argmax(n.query(cat[k]))], phonemes[count])
    count = count+1
    
print()
    
rate, data = wavfile.read("3Test.wav")
seconds = (data.shape[0]/float(rate))
N = int((seconds/6)*rate)
cat = []

for i in range(6):
    subdata = data[(i)*N:(i+1)*N-1]
    F=abs(fftpack.fft(subdata))
    f = f[:len(F)/2]
    F = F[:len(F)/2]
    F = F[100:900]
    F = F/100000
    cat.append(F)
cat =  np.array(cat)

for k in range(6):
    print(phonemes[np.argmax(n.query(cat[k]))], phonemes[count])
    count = count+1



()
('ou', 'i')
('er', 'ee')
('er', 'oo')
('er', 'oo')
('u', 'ai')
('oi', 'oi')
('u', 'i')
()
('er', 'e')
('er', 'er')
('oor', 'oor')
('u', 'air')
('ere', 'ere')
('er', 'ou')
()
('u', 'a')
('u', 'u')
('o', 'o')
('ar', 'ar')
('u', 'ow')
('ou', 'ou')
